In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from util.utilities import train, evaluate
from sklearn.model_selection import train_test_split
import os

# import all the models from models module
from models import VanillaRNN

In [ ]:
# preporcessing data

In [ ]:
# Split the data
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, shuffle=False)

In [39]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cpu


In [ ]:
# Create Batches with DataLoaders
batch_size = 32
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
input_size = # input size
output_size = 1
model = MLP(input_size, output_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs = 10

for epoch in range(num_epochs):
    train_loss, avg_train_loss = train(model, train_loader, criterion, optimizer)
    val_loss, avg_val_loss = evaluate(model, val_loader, criterion)

    # print every num times epoch only
    num = 1
    if ((epoch+1) % num == 0) or epoch == 0:
      print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.5f}, Average Training Loss: {avg_train_loss:.5f}, Validation Loss: {val_loss:.5f}, Average Validation Loss: {avg_val_loss:.5f}')